In [33]:
# This macro helps optimize the innermost loop in the kernel on your machine. You should not need to use it anywhere else
using LoopVectorization: @turbo

# This function takes in matrices ABC and adds B times C to the matrix A
function add_to_A_B_times_C!(A, B, C)
    @turbo for j in axes(C, 2)
        for k in axes(B, 2)
            for i in axes(A, 1)
                A[i, j] += B[i, k] * C[k, j]
            end
        end
    end
end

add_to_A_B_times_C! (generic function with 2 methods)

In [34]:
# This function takes in matrices ABC and adds B times C to the matrix A
# It uses blocking into blocks of size bks
function add_to_A_B_times_C!(A, B, C, bks)
    for i in 1:bks:size(A, 1)
        for j in 1:bks:size(C, 2)
            for k in 1:bks:size(B, 2)
                i_block_end = min(i+bks-1, size(A, 1))
                j_block_end = min(j+bks-1, size(C, 2))
                k_block_end = min(k+bks-1, size(B, 2))

                # Extracting the blocks
                A_block = view(A, i:i_block_end, j:j_block_end)
                B_block = view(B, i:i_block_end, k:k_block_end)
                C_block = view(C, k:k_block_end, j:j_block_end)

                # Perform block multiplication
                add_to_A_B_times_C!(A_block, B_block, C_block)
            end
        end
    end
end


add_to_A_B_times_C! (generic function with 2 methods)

In [36]:
function oblivious_add_to_A_B_times_C!(A, B, C, bks)
    i_size, j_size, k_size = size(A, 1), size(C, 2), size(B, 2)

    if min(i_size, j_size, k_size) <= bks
        add_to_A_B_times_C!(A, B, C)
    else

        i_mid = div(i_size, 2)
        j_mid = div(j_size, 2)
        k_mid = div(k_size, 2)

        for i1 in 1:i_mid
            for j1 in 1:j_mid
                for k1 in 1:k_mid
                    
                    i2 = i1 + i_mid - 1
                    j2 = j1 + j_mid - 1
                    k2 = k1 + k_mid - 1

                    oblivious_add_to_A_B_times_C!(
                        view(A, i1:i2, j1:j2),
                        view(B, i1:i2, k1:k2),
                        view(C, k1:k2, j1:j2),
                        bks
                    )
                end
            end
        end
    end
end


oblivious_add_to_A_B_times_C! (generic function with 2 methods)

In [ ]:
# Testing matmul function
A = zeros(2014, 301)
B = randn(2014, 1037)
C = randn(1037, 301)
println("Testing for memory allocation of oblivious matmul...")
allocated_memory = @ballocated oblivious_add_to_A_B_times_C!(A, B, C, 64)
@assert allocated_memory == 0
println("No memory allocated, good!")
println("Testing correctness of oblivious matmul...")
A .= 0; oblivious_add_to_A_B_times_C!(A, B, C, 64)
@assert A ≈ B * C
println("Result correct, good!")